In [5]:
import requests 
import json 
import urllib.request
import datetime
import time
from tqdm import tqdm 
import pandas as pd
import folium 
import webbrowser
from folium.plugins import MarkerCluster, MiniMap

In [39]:

def getMaskStoreInfo():
    url = "https://8oi9s0nnth.apigw.ntruss.com/corona19-masks/v1/storesByGeo/json" 
    req = requests.get(url) 
    counts = req.json()['count'] 
    addrs = [] 
    codes = [] 
    created_ats=[]
    latitudes = [] 
    longitudes = [] 
    names = [] 
    remain_stats=[]
    stock_ats=[]
    types = [] 
    req = requests.get(url) 
    json_data = req.json() 
    store_infos = json_data['stores'] 
    
    for i in range(len(store_infos)):  
        addrs.append(store_infos[i]['addr']) 
        codes.append(store_infos[i]['code']) 
        created_ats.append(store_infos[i]['created_at']) 
        latitudes.append(store_infos[i]['lat']) 
        longitudes.append(store_infos[i]['lng']) 
        names.append(store_infos[i]['name']) 
        remain_stats.append(store_infos[i]['remain_stat']) 
        stock_ats.append(store_infos[i]['stock_at']) 
        types.append(store_infos[i]['type'])  #정보를 문자열에 append
        
    mask_store_info_df = pd.DataFrame({"addr":addrs, "code":codes, "created_at":created_ats,"latitude":latitudes, "longitude":longitudes, "name":names, "remain_stat":remain_stats,"stock_at":stock_ats,"type":types}) 
    return mask_store_info_df #데이터프레임형태로 반환

        
def main():     
    my_info_df = getMaskStoreInfo()
    my_info_df.to_csv("mask_store_info.csv", index=False)
    data_for_draw = my_info_df.loc[:, ['name', 'latitude', 'longitude', 'remain_stat','addr']] 
    data_for_draw_nan = data_for_draw.dropna() 
    
    color_dic = {'plenty':'green', 'some':'orange', 'few':'red', 'empty':'gray', 'break':'black'}
    remain_stat_kor = {'plenty':'100개 이상', 'some':'30개 이상', "few":'2개이상 30개 미만', "empty":"재고 없음", "break":"문닫음" }
    
    
    map_hs = folium.Map((37.5028561, 127.0447712), zoom_start = 16) 
    mc = MarkerCluster() 
    names = list(data_for_draw_nan['name']) 
    latitudes = list(data_for_draw_nan['latitude']) 
    longitudes = list(data_for_draw_nan['longitude']) 
    remain_stats = list(data_for_draw_nan['remain_stat'])
    addrs = list(data_for_draw_nan['addr'])
    
    for i in tqdm(range(len(names))): 
        mc.add_child(folium.Marker(location = [latitudes[i], longitudes[i]], icon=folium.Icon(color=color_dic[remain_stats[i]]), popup='<'+names[i] +'> '+'\n'+addrs[i] + '\n' + remain_stat_kor[remain_stats[i]])) 
        map_hs.add_child(mc)

    map_hs.save('12.html')

   

In [40]:

    
    
if __name__ == "__main__":   
    main()

100%|████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 8891.65it/s]
